# Moisture and Temperature regressed with precipitation

## Import package

In [1]:
import h5py
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import product

## Load data

In [2]:
# Load ERA5 data
ERA5_PATH = "/work/b11209013/2024_Research/ERA5/"

q = {}; t = {}; w = {}

for central_lon in tqdm(range(0, 341, 20)):
    
    with xr.open_dataset(f"{ERA5_PATH}q/q_sub.nc", chunks={}) as f:
        lon_centered = (f["lon"] - central_lon + 180) % 360 - 180
        f = f.assign_coords(lon=lon_centered).sortby("lon")

        q[str(central_lon)] = f["q"].values*1000.0
    
    with xr.open_dataset(f"{ERA5_PATH}t/t_sub.nc", chunks={}) as f:
        lon_centered = (f["lon"] - central_lon + 180) % 360 - 180
        f = f.assign_coords(lon=lon_centered).sortby("lon")

        t[str(central_lon)] = f["t"].values

    with xr.open_dataset(f"{ERA5_PATH}w/w_Itp_sub.nc", chunks={}) as f:
        lon_centered = (f["lon"] - central_lon + 180) % 360 - 180
        f = f.assign_coords(lon=lon_centered).sortby("lon")

        w[str(central_lon)] = f["w"].mean(dim="lat").values

# Load IMERG time series data
IMERG_PATH = "/home/b11209013/2025_Research/Obs/Files/IMERG/Hovmoller.h5"

Hov = {}

with h5py.File(IMERG_PATH, "r") as f:
    lon = np.array(f.get("lon"))

    hov_grp = f.get("precip")

    Hov = {key: np.array(hov_grp.get(key)) for key in hov_grp.keys()}

100%|██████████| 18/18 [07:01<00:00, 23.42s/it]


## Compute regression

### Define functions

In [3]:
import numpy as np

def regression_slope(x, y):
    """Compute regression slope of y onto 1D x, vectorized over all grid points.
    
    Parameters
    ----------
    x : array_like, shape (nt,)
        Predictor time series.
    y : array_like, shape (nt, ...) 
        Predictand field with the same time dimension as x. Can be 2D, 3D, etc.
    
    Returns
    -------
    slope : ndarray, shape y.shape[1:]
        Regression slope at each grid point, with NaNs where regression is not defined.
    """
    # Convert to arrays
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)

    # Basic checks
    if x.ndim != 1:
        # Allow x to be (nt, 1, ..., 1) but collapse it
        x = x.reshape(x.shape[0], -1)
        if x.shape[1] != 1:
            raise ValueError("This implementation assumes x is a 1D time series (nt,).")
        x = x[:, 0]

    if y.shape[0] != x.shape[0]:
        raise ValueError("Time dimension of x and y must match (x.shape[0] == y.shape[0]).")

    nt = x.shape[0]
    # Flatten spatial dims of y: (nt, npoints)
    y_flat = y.reshape(nt, -1)                      # (nt, M)
    M = y_flat.shape[1]

    # Broadcast x to shape (nt, M)
    x2d = x[:, None]                                # (nt, 1)

    # Valid (non-NaN) mask per time & point
    mask = ~np.isnan(y_flat) & ~np.isnan(x2d)       # (nt, M)

    # Number of valid samples per point
    n = np.sum(mask, axis=0)                        # (M,)

    # Zero out invalid entries for sums
    x_masked = np.where(mask, x2d, 0.0)             # (nt, M)
    y_masked = np.where(mask, y_flat, 0.0)          # (nt, M)

    # Sums over time for each grid point
    sum_x  = np.sum(x_masked, axis=0)               # (M,)
    sum_y  = np.sum(y_masked, axis=0)               # (M,)
    sum_xx = np.sum(x_masked * x_masked, axis=0)    # (M,)
    sum_xy = np.sum(x_masked * y_masked, axis=0)    # (M,)

    # Closed-form slope per grid point
    denom = n * sum_xx - sum_x**2
    numer = n * sum_xy - sum_x * sum_y

    slope_flat = np.full(M, np.nan, dtype=float)
    valid_reg = (n > 1) & (denom != 0.0)
    slope_flat[valid_reg] = numer[valid_reg] / denom[valid_reg]

    # Back to original spatial shape
    slope = slope_flat.reshape(y.shape[1:])
    return slope


### Compute regression

In [4]:
# Compute regression
q_reg = {}
t_reg = {}
w_reg = {}

for q_key in q.keys():
    # preallocate dict
    q_reg[str(q_key)] = {}
    t_reg[str(q_key)] = {}
    w_reg[str(q_key)] = {}

    # find longitude index
    lon_idx = np.argmin(np.abs(lon - (int(q_key))))

    for hov_key in tqdm(Hov.keys(), desc=f"Processing LW key {q_key}"):
        Hov_ts = Hov[hov_key][:, lon_idx]

        q_reg[str(q_key)][str(hov_key)] = regression_slope(Hov_ts[:,None,None], q[q_key])
        t_reg[str(q_key)][str(hov_key)] = regression_slope(Hov_ts[:,None,None], t[q_key])
        w_reg[str(q_key)][str(hov_key)] = regression_slope(Hov_ts[:,None,None], w[q_key])

Processing LW key 340: 100%|██████████| 7/7 [00:14<00:00,  2.03s/it]


### Compute average values

In [5]:
q_reg_composite = {
    hov_key: np.nanmean(
        np.array([q_reg[q_key][hov_key] for q_key in q.keys()]),
        axis=0
    )
    for hov_key in Hov.keys()
}

t_reg_composite = {
    hov_key: np.nanmean(
        np.array([t_reg[t_key][hov_key] for t_key in t.keys()]),
        axis=0
    )
    for hov_key in Hov.keys()
}

w_reg_composite = {
    hov_key: np.nanmean(
        np.array([w_reg[w_key][hov_key] for w_key in w.keys()]),
        axis=0
    )
    for hov_key in Hov.keys()
}

## Save File

In [6]:
with h5py.File("/work/b11209013/2025_Research/regression/IMERG_ERA5.h5", "w") as f:
    q_grp = f.create_group("q")

    for hov_key in Hov.keys():
        q_grp_hov = q_grp.create_group(str(hov_key))
        for q_key in q.keys():
            q_grp_hov.create_dataset(str(q_key), data=np.array(q_reg[str(q_key)][str(hov_key)]))

    t_grp = f.create_group("t")

    for hov_key in Hov.keys():
        t_grp_hov = t_grp.create_group(str(hov_key))
        for t_key in t.keys():
            t_grp_hov.create_dataset(str(t_key), data=np.array(t_reg[str(t_key)][str(hov_key)]))

    w_grp = f.create_group("w")

    for hov_key in Hov.keys():
        w_grp_hov = w_grp.create_group(str(hov_key))
        for w_key in w.keys():
            w_grp_hov.create_dataset(str(w_key), data=np.array(w_reg[str(w_key)][str(hov_key)]))

    q_comp_grp = f.create_group("q_composite")

    for hov_key in Hov.keys():
        q_comp_grp.create_dataset(str(hov_key), data=np.array(q_reg_composite[str(hov_key)]))

    t_comp_grp = f.create_group("t_composite")

    for hov_key in Hov.keys():
        t_comp_grp.create_dataset(str(hov_key), data=np.array(t_reg_composite[str(hov_key)]))

    w_comp_grp = f.create_group("w_composite")

    for hov_key in Hov.keys():
        w_comp_grp.create_dataset(str(hov_key), data=np.array(w_reg_composite[str(hov_key)]))